In [1]:
from ast import literal_eval
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import gensim.models.word2vec
import itertools
import numpy as np
import re
import json
from math import log
from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer

from gensim.models.keyedvectors import KeyedVectors
path = lambda x : '~/.kaggle/competitions/quora-question-pairs/' + x

In [10]:
train1 = pd.read_csv(path('train.csv'))
cleaned_train = pd.read_csv('~/PycharmProjects/quora-question-pairs/source/data/newtrain.csv')
cleaned_train['question1'] = cleaned_train['question1'].apply(literal_eval)
cleaned_train['question2'] = cleaned_train['question2'].apply(literal_eval)

In [3]:
train.dropna(subset=['question1', 'question2'], inplace=True)

In [14]:
# cleaning data
def my_tokenize(s, stopword=stopword):
    s = s.lower() # put into lower case form
    s = re.sub(r"[\'\"\?.()/:,]", ' ',s) # remove certain punctuations and replace it with space
    s = re.sub(r"\s+",' ',s) # remove extra space
    s = re.sub(r"not\s+",'not_', s) # deal with negation
    tokens = word_tokenize(s) # split string into words (tokens)
    porter = PorterStemmer() 
    tokens = [porter.stem(t) for t in tokens] # put words into base form
    tokens = [t for t in tokens if t not in set(stopword)] # remove stopwords
    return tokens

In [5]:
def clean_data(df):
    stopword = stopwords.words('english')
    stopword = [e for e in stopword if e not in ('what', 'when', 'where','whether','which','whose', 'why','should','how')]
    contractions = json.load(open('../data/contractions.json', 'r+'))
    # spread out contraction for question1 and question2
    question1 = []
    for s1 in df['question1']:
        tokens = re.findall(r'\S+', s1)
        mylist= []
        for t in tokens:
            if t in contractions:
                t = contractions[t]
            mylist.append(t)
            result1 = " ".join(mylist)
        question1.append(result1)
       


    question2 = []
    for s2 in df['question2']:
        tokens = re.findall(r'\S+', s2)
        mylist= []
        for t in tokens:
            if t in contractions:
                t = contractions[t]
            mylist.append(t)
            result2 = " ".join(mylist)
        question2.append(result2)

    myquestion1 = []
    myquestion2 = []
    for s1, s2 in zip(df['question1'], df['question2']):
        myresult1 = my_tokenize(s1, stopword)
        myresult2 = my_tokenize(s2, stopword)
        myquestion1.append(myresult1)
        myquestion2.append(myresult2)
    df['question1']= myquestion1
    df['question2']= myquestion2
    return df

In [6]:
train = clean_data(train)

In [3]:
model = KeyedVectors.load('/Users/John/PycharmProjects/quora-question-pairs/source/data/gensimModel')

In [4]:
def gensim_load_vec(path="https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"):
    #use gensim_emb.wv.index2word if used this way to load vectors
    #gensim_emb = gensim.models.word2vec.Word2Vec.load(path)
    gensim_emb =  gensim.models.KeyedVectors.load_word2vec_format(path, binary=True,limit=500000)
    vocab = gensim_emb.index2word
    shape = gensim_emb.syn0.shape
    return gensim_emb, vec, shape, vocab

def map_word_frequency(document):
    return Counter(itertools.chain(*document))

def sentence2vec(tokenised_sentence_list, embedding_size, word_emb_model, a = 1e-3):
    sentence_vecs = []
    sentence_set=[]
    word_counts = map_word_frequency(tokenised_sentence_list)
    for sentence in tokenised_sentence_list:
        vs = np.zeros(embedding_size)
        sentence_length = len(sentence)
        for word in sentence:
            a_value = a / (a + word_counts[word]) # smooth inverse frequency, SIF
        try:
            vs = np.add(vs, np.multiply(a_value, word_emb_model[word])) # vs += sif * word_vector
        except Exception as e:
            pass
        #vs = np.log(vs) - np.log(sentence_length)
        vs = np.divide(vs, sentence_length) # weighted average
        sentence_set.append(vs)
    return sentence_set

In [ ]:
gensim_emb, vec, shape, vocab = gensim_load_vec()
gensim_emb.save('/Users/John/PycharmProjects/quora-question-pairs/source/data/gensimModel')

In [5]:
sent_list=[]
for q1, q2 in zip(cleaned_train['question1'], cleaned_train['question2']):
    try:
        token1 = word_tokenize(" ".join(q1))
        token2 = word_tokenize(" ".join(q2))
        sent_list.append(token1)
        sent_list.append(token2)
    except Exception as e:
        print(q1)
        print(q2)

In [7]:
sent_emb = sentence2vec(sent_list, 300 , model)

/Users/John/Dropbox/ECS171/HW3/venv/lib/python3.6/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in true_divide
/Users/John/Dropbox/ECS171/HW3/venv/lib/python3.6/site-packages/ipykernel_launcher.py:26: RuntimeWarning: divide by zero encountered in true_divide


In [8]:
y_pred = pd.DataFrame({'id' : [], 'is_duplicate': []})

is_dup = []
for i in range(0, 1000, 2):
    try:
        is_dup.append(float(cosine_similarity([sent_emb[i]],[sent_emb[i + 1]])))
    except Exception as e:
        print([sent_emb[i]])
        print([sent_emb[i]])
        print(cosine_similarity([sent_emb[i]],[sent_emb[i + 1]]))

In [6]:
len(sent_list)

808580

In [7]:
train.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,"[what, step, step, guid, invest, share, market...","[what, step, step, guid, invest, share, market]",0
1,1,3,4,"[what, stori, kohinoor, koh-i-noor, diamond]","[what, would, happen, indian, govern, stole, k...",0
2,2,5,6,"[how, increas, speed, internet, connect, use, ...","[how, internet, speed, increas, hack, dn]",0
3,3,7,8,"[whi, mental, veri, lone, how, solv]","[find, remaind, when, [, math, ], 23^, {, 24, ...",0
4,4,9,10,"[which, one, dissolv, water, quikli, sugar, sa...","[which, fish, would, surviv, salt, water]",0


In [15]:
stopword = set(stopwords.words('english'))
stopword_exceptions = set(['what', 'when', 'where', 'whether', 'which', 'whose', 'why', 'should', 'how'])
stopword = stopword.difference(stopword_exceptions)
train1['question1'].apply(my_tokenize)

AttributeError: 'float' object has no attribute 'lower'